In [24]:
from __future__ import division
import pandas as pd
import numpy as np
import json
import os,sys
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np

In [25]:
dataset_base_dir = '../datasets/students-dataset/'

In [26]:
def convert_object_type_to_category(df):
    """Converts columns of type object to category."""
    df = pd.concat([df.select_dtypes(include=[], exclude=['object']),
                  df.select_dtypes(['object']).apply(pd.Series.astype, dtype='category')
                  ], axis=1).reindex(df.columns, axis=1)
    return df

In [27]:
def remove_id_cols(df):
    """Removes columns that have 'id' in their name."""
    return df[[col for col in df.columns if 'id' not in col]]

In [28]:
TRAIN_FILE = os.path.join(dataset_base_dir, 'train.csv')

with open(TRAIN_FILE, 'r') as f:
    train_df = pd.read_csv(f)

target = "score_MAT"
columns = train_df.columns
train_df = convert_object_type_to_category(train_df)

/home/cip/miniconda3/envs/arl_uva/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (16,17,18,19,24,25,38,61,112,113,123,127,136) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [29]:
train_df = remove_id_cols(train_df)
print(train_df.columns)

Index(['score_MAT', 'level_MAT', 'score_LEN', 'level_LEN', 'score_ING',
       'level_ING', 's_gender', 's_birth_year', 's_is_living_with_mother',
       's_is_living_with_father',
       ...
       't_number_of_special_attention_students',
       't_extent_of_student_involvement_during_class',
       't_extent_of_teaching_methods_variety', 't_extent_of_opinion_on_school',
       't_extent_of_class_behaviour', 't_extent_of_resource_variety',
       't_extent_of_good_work_by_non_teachers', 't_number_of_subjects_taught',
       't_extent_of_satisfaction_job_and_school',
       't_extent_of_results_satisfaction'],
      dtype='object', length=149)


In [30]:
col_list_train = train_df.columns.values
temp_dict = train_df.describe().to_dict()
mean_std_dict = {}
for key, value in temp_dict.items():
    mean_std_dict[key] = [value['mean'], value['std']]

output_file_path = os.path.join(dataset_base_dir, 'mean_std.json')
with open(output_file_path, 'w') as f:
    json.dump(mean_std_dict, f)

print("Mean and std saved to {}".format(output_file_path))

Mean and std saved to ../datasets/students-dataset/mean_std.json


In [31]:
cat_cols = train_df.select_dtypes(include='category').columns
vocab_dict = {}
for col in cat_cols:
    vocab_dict[col] = list(set(train_df[col].cat.categories)-{"?"})

output_file_path = os.path.join(dataset_base_dir,'vocabulary.json')
with open(output_file_path, mode="w") as output_file:
    output_file.write(json.dumps(vocab_dict))
    output_file.close()
print(vocab_dict)

{'s_gender': ['FEMALE', 'MALE'], 's_is_living_with_mother': [False, True], 's_is_living_with_father': [False, True], 's_is_living_with_siblings': [True], 's_is_living_with_other_relatives': [True], 's_has_repeated': [False, True], 's_birth_country': ['AUT', 'POL', 'BGD', 'ESP', 'MLI', 'MRT', 'SVN', 'CHE', 'CHL', 'CZE', 'PRT', 'HND', 'USA', 'PRI', 'BOL', 'SEN', 'FIN', 'ESH', 'GBR', 'GEO', 'ARG', 'PHL', 'SWE', 'IRL', 'ROU', 'CPV', 'DEU', 'COG', 'CHN', 'CMR', 'VEN', 'FRA', 'ISL', 'PER', 'NLD', 'CRI', 'COL', 'BGR', 'BEL', 'NGA', 'GMB', 'LTU', 'BLR', 'DNK', 'ECU', 'CAN', 'IND', 'SLV', 'MCO', 'PRY', 'BRA', 'NPL', 'ITA', 'SYR', 'HUN', 'SVK', 'CUB', 'PAK', 'ETH', 'URY', 'DOM', 'GIN', 'RUS', 'IRN', 'MEX', 'BIH', 'LVA', 'DZA', 'MAR', 'GTM', 'PAN', 'UKR', 'GNQ'], 's_nazionality_country': ['AUT', 'POL', 'SRB', 'MLI', 'MRT', 'BGD', 'ESP', 'SVN', 'CHL', 'CZE', 'LKA', 'USA', 'HND', 'PRT', 'BOL', 'SEN', 'FIN', 'GBR', 'GEO', 'ARG', 'PHL', 'SWE', 'IRL', 'ROU', 'CPV', 'DEU', 'COG', 'CHN', 'UKR', 'KEN', '

In [32]:
stats = {}
stats["feature_names"] = list(train_df.columns)
stats["mean_std"] = mean_std_dict
stats["sensitive_column_names"] = ["s_gender", "f_has_been_recommended_school"]
stats["sensitive_column_values"] = ["Female", "False"]
stats["target_column_name"] = "s_has_repeated"
stats["target_column_positive_value"] = "True"
stats["vocabulary"] = vocab_dict

output_file_path = os.path.join(dataset_base_dir, 'dataset_stats.json')
with open(output_file_path, mode="w") as output_file:
    output_file.write(json.dumps(stats, indent=4, sort_keys=True))
    output_file.close()

In [33]:
# save train_df to csv
output_file_path = os.path.join(dataset_base_dir, 'train_processed.csv')
train_df.to_csv(output_file_path, index=False)
print("Processed train data saved to {}".format(output_file_path))

Processed train data saved to ../datasets/students-dataset/train_processed.csv
